# Preprocessing #
Evan Petrimoulx\
Sania Palaha\
Data Analytics 1

In [1]:
import os
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.image import imread
from numpy.fft import fft2, fftshift
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam


In [2]:
image_pixel_array = [] 

for file in os.listdir('/Users/saniapalaha/Desktop/Plant_leave_diseases_dataset_without_augmentation'):
    if file.endswith(".jpg"):
        image = imread(file)
        image_pixel_array.append(image)

image_pixel_array = np.array(image_pixel_array)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/saniapalaha/Desktop/Plant_leave_diseases_dataset_without_augmentation'

Dataset is now loaded and stored as a 3D numpy array (image, (horizontal pixels, vertical pixels)). Now we can perform the data preprocessing, and store the dataset.

In [3]:
# Fourier Transform Information
Fourier_Data = []
num_images = image_pixel_array.shape[0]

for image in num_images:
    image_FT = np.fft2(image_pixel_array[image]) # Compute 2D Fourier Transform
    Fourier_Data.append(image_FT)                # Append FT data to the FT_Data list. This will be added as a column in the Dataframe


# Proof of concept: (Can remove)


plt.figure(figsize=(12, 8))
plt.imshow(Fourier_Data[0])
plt.show()
print(image_pixel_array.shape)

AttributeError: 'list' object has no attribute 'shape'